# Gráfico de time varying IRFs en diferentes horizontes de tiempo 

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
import numpy as np
import seaborn as sns
import os

## Horizontes con shock findex 10k

In [6]:
# Archivo Excel
file = 'Horizon IRFs.xlsx'

# Obtener todas las hojas
sheets = pd.ExcelFile(file).sheet_names

# Crear carpeta de salida si no existe
output_dir = "graficos_irf"
os.makedirs(output_dir, exist_ok=True)

# Función auxiliar para convertir trimestres a fechas
def quarter_to_date(qstr):
    year, q = qstr.lower().split('q')
    month = {'1': 1, '2': 4, '3': 7, '4': 10}[q]
    return pd.Timestamp(f'{year}-{month:02d}-01')

# Estilo general
sns.set_style("whitegrid")
plt.rcParams.update({
    'font.size': 11,
    'axes.titlesize': 13,
    'axes.edgecolor': 'black',
    'axes.linewidth': 1,
    'grid.color': 'white',
    'axes.grid': False,
    'figure.dpi': 150,
})

# Títulos personalizados
titulos = [
    'Estabilidad Bancaria',
    'Concentración Bancaria',
    'Inflación',
    'Consumo',
    'Inversión',
    'Producto'
]

In [10]:
# Procesar cada hoja
for sheet in sheets:
    df = pd.read_excel(file, sheet_name=sheet)

    # Extraer columnas
    variables = df['VARIABLES']
    horizons = pd.to_numeric(df['HORIZON'], errors='coerce')

    # Filtrar filas válidas
    valid_rows = ~horizons.isna()
    variables = variables[valid_rows]
    horizons = horizons[valid_rows].astype(int)
    data = df.loc[valid_rows, df.columns[2:]]

    # Convertir columnas de fechas
    time_labels = [quarter_to_date(q) for q in data.columns]

    # Variables y horizontes únicos
    unique_vars = variables.unique()
    unique_horizons = sorted(horizons.unique())

    # Configuración de subplots
    n_rows, n_cols = 3, 2
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(10, 8), sharex=False)
    axes = axes.flatten()

    # Paleta de colores
    cmap = plt.get_cmap('Set2')
    colors = [cmap(i) for i in range(len(unique_horizons))]

    # Graficar
    for idx, var in enumerate(unique_vars):
        ax = axes[idx]

        for h_idx, h in enumerate(unique_horizons):
            mask = (variables == var) & (horizons == h)
            irf_series = data[mask].values.flatten()
            ax.plot(time_labels, irf_series, color=colors[h_idx], linewidth=1.1)
        
        ax.set_title(titulos[idx], fontsize=12, fontweight='bold', pad=8)
        ax.axhline(0, color='black', linewidth=1)
        ax.tick_params(axis='x', labelrotation=0, labelsize=10)
        ax.tick_params(axis='y', labelsize=10)
        ax.set_xlim([pd.to_datetime(time_labels[0]), pd.to_datetime(time_labels[-1])])
        ax.set_xlabel('')
        ax.set_ylabel('')
        ax.set_xticks(time_labels[::12])
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

    # Eliminar subplots vacíos
    for j in range(len(unique_vars), n_rows * n_cols):
        fig.delaxes(axes[j])

    # Ajustar diseño y guardar
    plt.tight_layout(pad=2.0)
    output_path = os.path.join(output_dir, f"{sheet}.png")
    plt.savefig(output_path, bbox_inches='tight')
    plt.close()

print("Gráficos exportados correctamente.")

Gráficos exportados correctamente.


In [37]:
# Leer archivo y hojas
file = 'Horizon IRFs.xlsx'
sheets = ['H_Findex', 'H_Acceso', 'H_Uso', 'H_Prof']

# Función para convertir trimestres a fechas
def quarter_to_date(qstr):
    year, q = qstr.lower().split('q')
    month = {'1': 1, '2': 4, '3': 7, '4': 10}[q]
    return pd.Timestamp(f'{year}-{month:02d}-01')

# Estilo elegante
sns.set_style("whitegrid")
plt.rcParams.update({
    'font.size': 9,
    'axes.titlesize': 10,
    'axes.edgecolor': 'black',
    'axes.linewidth': 1,
    'grid.color': 'white',
    'axes.grid': False,
    'figure.dpi': 150,
})

# Títulos por variable del sistema (columnas)
titulos_columnas = [
    'Estabilidad Bancaria',
    'Concentración Bancaria',
    'Inflación',
    'Consumo',
    'Inversión',
    'Producto'
]

# Títulos por shocks (filas)
titulos_filas = ['Findex', 'Acceso', 'Uso', 'Profundidad']

# Paleta de colores
cmap = plt.get_cmap('Set2')
colors = [cmap(i) for i in range(5)]  # 5 horizontes

# Crear figura 4x6
fig, axes = plt.subplots(4, 6, figsize=(17, 9), sharex=False)
axes = axes.reshape(4, 6)

# Recorrer hojas (shocks)
for row_idx, (sheet, shock_name) in enumerate(zip(sheets, titulos_filas)):
    df = pd.read_excel(file, sheet_name=sheet)

    # Extraer columnas
    variables = df['VARIABLES']
    horizons = pd.to_numeric(df['HORIZON'], errors='coerce')

    # Filtrar filas válidas
    valid_rows = ~horizons.isna()
    variables = variables[valid_rows]
    horizons = horizons[valid_rows].astype(int)
    data = df.loc[valid_rows, df.columns[2:]]
    
    time_labels = [quarter_to_date(q) for q in data.columns]
    unique_vars = variables.unique()
    unique_horizons = sorted(horizons.unique())

    # Recorrer variables (columnas)
    for col_idx, var in enumerate(unique_vars):
        ax = axes[row_idx, col_idx]

        for h_idx, h in enumerate(unique_horizons):
            mask = (variables == var) & (horizons == h)
            irf_series = data[mask].values.flatten()
            ax.plot(time_labels, irf_series, color=colors[h_idx], linewidth=1.1)

        ax.axhline(0, color='black', linewidth=1)
        ax.tick_params(axis='x', labelrotation=0, labelsize=10)
        ax.tick_params(axis='y', labelsize=10)
        ax.set_xlim([time_labels[0], time_labels[-1]])
        ax.set_xticks(time_labels[::17])
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

        # Título de columna en la primera fila
        if row_idx == 0:
            ax.set_title(titulos_columnas[col_idx], fontsize=14, fontweight='bold', pad=10)

        # Etiqueta de fila en la primera columna
        if col_idx == 0:
            ax.set_ylabel(shock_name, fontsize=14, fontweight='bold', labelpad=8)

# Añadir leyenda personalizada solo en el primer subplot
# Añadir leyenda general al final de la figura
handles = [plt.Line2D([0], [0], color=colors[i], lw=2) for i in range(len(unique_horizons))]
labels = [f'H = {h}' for h in unique_horizons]
fig.legend(handles, labels, fontsize=13, loc='lower center', ncol=len(unique_horizons), bbox_to_anchor=(0.5, -0.03))

# Ajustar diseño y exportar
plt.tight_layout(pad=2.0)
plt.savefig("Horizon_total.png", bbox_inches='tight')
plt.close()

print("Gráfico 4x6 exportado como 'Horizon_total.png'")

Gráfico 4x6 exportado como 'Horizon_total.png'
